In [1]:
import os
import time
import numpy as np
import pandas as pd

from skimage import io

import matplotlib.pyplot as plt

from CellSeg.segmentation import Segmentation
from CellSeg.analysis import cell_analysis, edge_analysis, face_analysis
%load_ext snakeviz

In [2]:
%pdb

Automatic pdb calling has been turned ON


In [3]:
path = "../data/ctrl/"
seg_filename = "ResultofC1-Composite_cp_masks-edited.tif"

pixel_size = dict(x_size = 0.183,
                  y_size = 0.183,
                  z_size = 0.5,)

In [4]:

image = io.imread(os.path.join(path, seg_filename))
seg = Segmentation(image, pixel_size, path)

In [5]:
%%snakeviz
start_time = time.time()
seg.perform_prerequisite()
# print("--- %s seconds ---" % (time.time() - start_time))

This folder npz is not empty, 
If you want to save file here, turn overright to True
 
*** Profile stats marshalled to file '/tmp/tmpzvt47ao_'.
Embedding SnakeViz in this document...


In [6]:
%%snakeviz
start_time = time.time()
seg.cell_segmentation()
# print("--- %s seconds ---" % (time.time() - start_time))
# 168 sec

 
*** Profile stats marshalled to file '/tmp/tmpistsrp3r'.
Embedding SnakeViz in this document...


In [7]:
%%snakeviz
start_time = time.time()
seg.edge_segmentation()
# print("--- %s seconds ---" % (time.time() - start_time))
#2.09e+3

 
*** Profile stats marshalled to file '/tmp/tmpcp95777u'.
Embedding SnakeViz in this document...


In [8]:
%%snakeviz
start_time = time.time()
seg.face_segmentation()
# print("--- %s seconds ---" % (time.time() - start_time))

 
*** Profile stats marshalled to file '/tmp/tmpm8d9xlis'.
Embedding SnakeViz in this document...


In [5]:
%%snakeviz
cell_analysis(seg)
# 1sec

 
*** Profile stats marshalled to file '/tmp/tmp3g3njwq4'.
Embedding SnakeViz in this document...


In [11]:
%%snakeviz
edge_analysis(seg)

 
*** Profile stats marshalled to file '/tmp/tmpt4z0xyu9'.
Embedding SnakeViz in this document...


In [5]:
%%snakeviz
face_analysis(seg)

 
*** Profile stats marshalled to file '/tmp/tmpvtkllpbp'.
Embedding SnakeViz in this document...
